# <span style="color:green">Oregon Hydrographic Area Historical ET & Consumptive Use Geodatabase: OWRD, DRI, and OpenET</span>

<center><img src="data_viz_et.png" width="900"/></center>
<!-- ![Data Viz](data_viz_et.png "ET/CU Geodatabase") -->

______________________
<br>


# ET/CU field-level data visualization & retrieval [tool](https://dri-apps.projects.earthengine.app/view/owrd-oregon-etcu-field-summaries)
> Retrieve monthly/annual timeseries data for individual field boundaries for 1985-2024

<center><img src="data_viz_field_level_tool.png" width="1200"/></center>

<br>

# Python-based ET/CU data aggregation notebook
> Python workflow to summarize 40 years (1985-2024) of monthly/annual data variables for watershed or irrigation district boundaries

## Data Variable Definitions:
* OpenET ensemble actual ET ($ET_{a}$)
* Bias-corrected gridMET reference ET ($ET_{o}$)
* Effective precipitation ($P_{rz}$)
* Total precipitation (PPT)
* Crop-specific potential ET ($ET_{c}$)
* Consumptive use (CU)
* Net irrigation water requirement (NIWR)
* Applied water (AW)


## <span style="color:red">Please run this notebook using a local Python environment. NOTE: this notebook will download the entire Oregon geodatabase (11 GB zipped, 30GB unzipped) to the local system.</span>

___

# Overview of Walkthrough:

INTRODUCTION_________________________________

1. Import Python packages
    - [geopandas](https://geopandas.org/en/stable/#) is a python-based geospatial analysis tool.
    - [pandas](https://pandas.pydata.org/) is a python-based data analysis and manipulation tool.
    - [polars](https://pola.rs/) is an efficient/fast python-based data analysis and manipulation tool.
    - [requests](https://pypi.org/project/requests/) is a simple, yet elegant, HTTP library.
    - [py7zr](https://pypi.org/project/py7zr/) supports 7zip archive compression, decompression, encryption and decryption.
    - [folium](https://python-visualization.github.io/folium/latest/index.html) is a python-based interactive geospatial analysis and visualization tool. Avoids the use of Earth Engine and allows us to visualize locally-stored geospatial data and the geodatabase.
2. Download & Visualize the Historical ET/CU Geodatabase

TIMESERIES EXPORT_____________________________

3. Export spatially and temporally aggregated ET/CU data

DATA COMPARISON______________________________

4. Compare applied water estimates to total diversions for Tumalo Irrigation District
> monthly & annual plotting, canal diversion comparisons, and conveyance loss estimates

___
<br><br>

## 1. Import Python packages

In [19]:
import os
import geopandas as gpd
import pandas as pd
import polars as pl
import folium
import requests
from requests.auth import HTTPBasicAuth
import py7zr

In [ ]:
# define the path to the main directory
# root_directory = r'/Users/blakeminor/Documents/GitHub/OR_OWRD_OSU_training' # Mac
root_directory = r'C:\Users\bminor\Documents\GitHub\OR_OWRD_OSU_training' # Windows

## 2. Download & Visualize the Historical ET/CU Geodatabase
> Download the geodatabase programmatically using **requests** below<br><br>
> If you prefer a manual download, you can login with the username and passwor [here](http://crushftp.dri.edu/)<br>
> Once downloaded, move the 7-zip file into the "data" subfolder and then unzip the file<br>

In [17]:
# username and password required for the DRI CrushFTP URL
usernm = ''
passwrd = ''

In [ ]:
def download_zip_https(url, username, password, local_filename):
    try:
        # Use HTTP Basic Authentication
        r = requests.get(url, auth=HTTPBasicAuth(username, password), stream=True)
        r.raise_for_status() # Raise an exception for bad status codes

        with open(local_filename, 'wb') as f:
            for block in r.iter_content(1024):
                if not block:
                    break
                f.write(block)
        print(f"File '{local_filename}' downloaded successfully via HTTPS.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading file: {e}")

# geodatabase URL
server_url = "https://crushftp.dri.edu/preliminary_or_field_geopackage.7z"

# local path for storing the geodatabase
local_file_path = os.path.join(root_directory, 'data', 'preliminary_or_field_geopackage.7z')

if not os.path.isfile(local_file_path):
    print('Atemmpting to download the geodatabase from CrushFTP, please wait ')
    download_zip_https(server_url, usernm, passwrd, local_file_path)
else:
    print('Geodatabase has already been downloaded, skip to the next cell')

### Unzip the geodatabase if needed
> Unzip the geodatabase programmatically using **py7zr** below<br><br>
> To manually unzip the geodatabase, please use the [7-zip](https://www.7-zip.org/) application for Windows machines or the built-in Archive Utility application on MacOS/Linux machines

In [ ]:
# local path to the geodatabase
local_file_path = os.path.join(root_directory, 'data', 'preliminary_or_field_geopackage.7z')

if os.path.isfile(local_file_path):
    if not os.path.isfile(os.path.join(root_directory, 'data', 'preliminary_or_field_geopackage.gpkg')):
        try:
            with py7zr.SevenZipFile(local_file_path, 'r') as archive:
                archive.extractall(path=os.path.join(root_directory, 'data', 'preliminary_or_field_geopackage.gpkg'))
        except Exception as e:
            print(e)
    else:
        print('geodatabase has already been unzipped, skip to the next cell')
else:
    print('geodatabase zip file was not located, please download it using the previous code blocks or manually download it')

### Read the geodatabase
> We just want the geometries with ID's for determining which fields to include in spatial & temporal aggregations<br>
> Only need one layer of the geodatabase for this

In [ ]:
# read the file into memory
try:
    gdf = gpd.read_file(os.path.join(root_directory, 'data', 'preliminary_or_field_geopackage.gpkg'), 
                        layer='Oregon_Hyd_Area_Ag_Boundaries_20241016', columns=['OPENET_ID'])
    print(gdf)
except Exception as e:
    print(e)

### Read a shapefile to define the region used for aggregating the field-level data from the geodatabase

In [34]:
# first, define the shapefile name (example uses the Tumalo Irrigation District boundary)
region_shapefile_name = 'tumalo_irrigation_district'


In [ ]:
# read the file into memory
try:
    region_gdf = gpd.read_file(os.path.join(root_directory, 'data', f'{region_shapefile_name}.shp'))
    print(region_gdf)
except Exception as e:
    print(e)

### Use the region to process get the list of field ID's to process

In [ ]:
def subset_by_centroid_within_region(gdf_to_filter, gdf_regions):
    """
    Subsets a GeoDataFrame (gdf_to_filter) to keep only the geometries whose 
    centroids fall within the polygons of another GeoDataFrame (gdf_regions).

    Args:
        gdf_to_filter (gpd.GeoDataFrame): The GeoDataFrame to filter (e.g., polygons).
        gdf_regions (gpd.GeoDataFrame): The GeoDataFrame with region polygons.

    Returns:
        gpd.GeoDataFrame: The filtered GeoDataFrame.
    """
    # 1. Ensure both GeoDataFrames have the same CRS
    if gdf_to_filter.crs != gdf_regions.crs:
        print(f"Reprojecting the region CRS to match the geodatabase CRS (EPSG:2994, Oregon Lambert Conformal Conic): {gdf_to_filter.crs}")
        gdf_regions = gdf_regions.to_crs(gdf_to_filter.crs)

    # 2. Create a temporary GeoSeries of the centroids
    # Note: .centroid can fall outside the polygon for complex shapes, 
    # use .representative_point() if you need a point guaranteed to be inside.
    centroids = gdf_to_filter.centroid
    
    # Create a temporary GeoDataFrame for the join, setting centroids as the active geometry
    gdf_centroids = gdf_to_filter.copy()
    gdf_centroids['centroid_geometry'] = centroids
    gdf_centroids = gdf_centroids.set_geometry('centroid_geometry')
    
    # 3. Perform a spatial join with predicate='within'
    # We use an inner join to keep only records where a centroid is within a region
    # The 'left_index=True, right_index=True' is generally not needed here, sjoin handles it.
    join_result = gpd.sjoin(gdf_centroids, gdf_regions, how="inner", predicate="within")
    
    # The join result contains rows where centroids were within a region. 
    # The original geometry column ('geometry') is still present in the join result
    # We can now set the active geometry back to the original polygons
    filtered_gdf = join_result.set_geometry('geometry')
    
    # Drop the temporary centroid column and any extra columns from the join
    # (e.g., 'index_right' which is added by sjoin)
    filtered_gdf = filtered_gdf.drop(columns=['centroid_geometry', 'index_right'], errors='ignore')
    
    return filtered_gdf

# Example Usage:
# Assuming you have gdf_polygons and gdf_regions loaded
# filtered_data = subset_by_centroid_within_region(gdf_polygons, gdf_regions)
